# Bank Marketing Analysis Questionário

O objetivo deste notebook é responder de maneira clara 6 perguntas sobre o Bank Marketing Data Set disponível [aqui](https://archive.ics.uci.edu/ml/datasets/bank+marketing).

Foram utilizadas as seguintes bibliotecas para atingir tal objetivo:
 - pandas
 - numpy
 - plotly
  
Plotly é uma biblioteca multilinguagem para geração de gráficos. Sua API em Python necessita de criação de credenciais. O cadastro pode ser feito no próprio site da plotly e a licença gratuita dele permite a criação de até 25 gráficos distintos.  

## 1. Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


## 2. Credentials and Options

In [2]:
plotly.tools.set_credentials_file(username='alelopes', api_key='ZETQxILVg72WbsKtxKyy')
pd.options.mode.chained_assignment = None
init_notebook_mode(connected=True)


## 3. Reading The Data

In [3]:
bank = pd.read_csv("../data/bank-full.csv",sep=";",dtype='category')

In [4]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
bank['age']=bank['age'].astype("int32")
bank['duration']=bank['duration'].astype("int32")
bank['campaign']=bank['campaign'].astype("int32")
bank['pdays']=bank['pdays'].astype("int32")
bank['previous']=bank['previous'].astype("int32")
bank['balance']=bank['balance'].astype("int32")

## 3. Questions 


### 3.1 - Qual profissão tem mais tendência a fazer um empréstimo? De qual tipo?

Para responder essa questão, vamos fazer uma análise em cima das variáveis 'job', 'loan' e 'housing', criando novas variáveis e agrupando os resultados de interesse em torno da variável 'job'

In [6]:
jobs_and_loans = bank[['job','loan','housing']].copy()
jobs_and_loans['loan']=jobs_and_loans['loan'].eq('yes').mul(1)
jobs_and_loans['housing']=jobs_and_loans['housing'].eq('yes').mul(1)
jobs_and_loans['counts'] = 1
jobs_and_loans['loans'] = jobs_and_loans['loan'] | jobs_and_loans['housing'] 
agg=jobs_and_loans.groupby('job').sum()

Com esses resultados em mãos, vamos plotar um gráfico de barras de pessoas com algum tipo de empréstimo pela profissão dessa pessoa

In [7]:
data = [go.Bar(x=agg.index.get_level_values(0),
            y=agg.loans,name='People with Any kind of Loan'),
       go.Bar(x=agg.index.get_level_values(0),
            y=agg.counts,name='All Contacted People'),
       ]
layout = go.Layout(
    title='People Contacted with Loans and All Contacted by Jobs',
)
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='first-plot')

Analisando o gráfico acima, é possível ver que blue-collar é a profissão que mais teve pessoas contactadas e a profissão que teve maior número de pessoas com empréstimo em números absolutos. Mas e proporcionalmente? O próximo gráfico mostra a relação de pessoas com empréstimo por todas as contactadas por profissão.

In [8]:
data = [go.Bar(x=agg.index.get_level_values(0),
            y=agg.loans/agg.counts,name='Loans/All')
       ]
layout = go.Layout(
    title='Ratio of people with loan X all contacted people',
)
fig = go.Figure(data=data, layout=layout)

iplot(fig,filename='second_plot')

Com esse resultado, podemos concluir que **78%** das pessoas com profissão *blue-collar* possuem algum tipo de empréstimo. Mas de qual tipo? O gráfico abaixo plota os tipos de empréstimo por profissão:

In [9]:
data = [go.Bar(x=agg.index.get_level_values(0),
            y=agg.loan,name='Loan'),
        go.Bar(x=agg.index.get_level_values(0),
            y=agg.housing,name='Housing Loan')
       ]
layout = go.Layout(
    title='Types of Loans by Job',
)
fig = go.Figure(data=data, layout=layout)

iplot(fig,filename='third')

<nb>

Podemos concluir que *blue-collar* é a profissão que possui maior tendência de fazer um empréstimo sendo majoritoriamente do tipo imobiliário.

<nb>

### 3.2 - Fazendo uma relação entre número de contatos e sucesso da campanha quais são os pontos relevantes a serem observados?

Para responder essa questão vamos isolar as variáveis *previous, campaign* e *y* que representam o número de contatos realizados antes da campanha atual, o número de contatos realizados na campanha atual e o resultado da campanha respectivamentes.

In [10]:
previous_contacts = bank[['previous','campaign','y']].copy()

In [11]:
previous_contacts.groupby(['y']).describe()

campaign                                               previous            \
       count      mean       std  min  25%  50%  75%   max    count      mean   
y                                                                               
no   39922.0  2.846350  3.212767  1.0  1.0  2.0  3.0  63.0  39922.0  0.502154   
yes   5289.0  2.141047  1.921826  1.0  1.0  2.0  3.0  32.0   5289.0  1.170354   

                                          
          std  min  25%  50%  75%    max  
y                                         
no   2.256771  0.0  0.0  0.0  0.0  275.0  
yes  2.553272  0.0  0.0  0.0  1.0   58.0

Podemos perceber na tabela acima que o primeiro quartil, mediana e terceiro quartis tem valores iguais para os valores de y=*no* (y=0) e y=*yes* (y=1) na campanha atual. Como a média de ambos encontram-se na casa de 2.8 e 2.1 com desvio padrão de ambas sendo superior a diferença entre as médias dos y's, podemos concluir que analisando apenas o resultado da campanha, nada podemos afirmar em relação ao y.

Contudo, podemos analisar os contatos prévios e dividí-los em "ocorreu contato prévio" ou "nunca ocorreu contato prévio". Faremos isso a seguir:

In [12]:
previous_contacts['previous']=previous_contacts.previous>0
agg_prev_y =previous_contacts.groupby(['y','previous'],as_index=False).count()

In [13]:
ratios=[previous_contacts.groupby(['y','previous']).count().campaign[1]/previous_contacts.groupby(['y','previous']).count().campaign[0:2].sum(),previous_contacts.groupby(['y','previous']).count().campaign[3]/previous_contacts.groupby(['y','previous']).count().campaign[2:4].sum()]

Com tais dados, podemos plotar um gráfico de inscritos/não inscritos na campanha atual com relação a se eles foram contactados previamente ou nunca foram contactados: 

In [14]:
data = [go.Bar(x=['Never Contacted','Previously Contacted'],
            y=agg_prev_y[agg_prev_y['y']=='no'].campaign,name='Subscribed'),
        go.Bar(x=['Never Contacted','Previously Contacted'],
            y=agg_prev_y[agg_prev_y['y']=='yes'].campaign,name='Not Subscribed')
        
               ]
layout = go.Layout(
    title='Subscribed/Not Subscribed X Never Contacted/Contacted in Previous Campaigns',
)
fig = go.Figure(data=data, layout=layout)

iplot(fig,filename='fourth')

Podemos ver então que **23%** das pessoas contactadas previamentes se inscreveram na campanha e que pouco mais de **9%** das pessoas que nunca foram contactadas antes se inscreveram na campanha. Ou seja, a chance de uma pessoa préviamente contactada se inscrever/assinar a campanha é quase o dobro de quem nunca foi contactado anteriormente

<br>

### 3.3 - Baseando-se nos resultados de adesão desta campanha qual o número médio e o máximo de ligações que você indica para otimizar a adesão?

Para essa análise vamos isolar o número de ligações dessa campanha (*campaign*) e o resultado da campanha (*y*):

In [15]:
campaign_y = bank[['campaign','y']].copy()

Gerarei os quartis para saber se os dados estão mais concentrados com *outliers* ou espalhado:

In [16]:
campaign_y.groupby('y').quantile([0.6,0.7,0.8,0.9,0.95,0.98])

campaign
y                 
no  0.60       2.0
    0.70       3.0
    0.80       4.0
    0.90       6.0
    0.95       8.0
    0.98      13.0
yes 0.60       2.0
    0.70       2.0
    0.80       3.0
    0.90       4.0
    0.95       5.0
    0.98       8.0

Como vamos analisar o número médio e máximo de ligações para otimizar a adesão, retirar os *outliers* vai facilitar a análise. Setei então a análise a menos de 13 ligações, englobando algo perto do 98th percentil:

In [17]:
excluding_outliers=campaign_y[campaign_y.campaign<13]
excluding_outliers.campaign=excluding_outliers.campaign.astype('category')
positive_campaign=excluding_outliers[excluding_outliers['y']=='yes'].groupby('campaign').count()
negative_campaign=excluding_outliers[excluding_outliers['y']=='no'].groupby('campaign').count()

Agora, podemos gerar finalmente 2 gráficos. Um do número de inscritos/não inscritos na campanha pelo número de ligações feitas e a proporção de inscritos por total de contactados para cada número de ligações feitas:

In [18]:
trace1 = go.Scatter(x=positive_campaign.index.get_level_values(0),
            y=positive_campaign.y,name='subscribed',mode = 'lines+markers')


trace2 = go.Scatter(x=negative_campaign.index.get_level_values(0),
            y=negative_campaign.y,name='not subscribed',mode = 'lines+markers')

trace3=go.Scatter(x=negative_campaign.index.get_level_values(0),
            y=positive_campaign.y/(negative_campaign.y+positive_campaign.y),name='ratio',mode = 'lines+markers',yaxis='y2',
                      line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4
        )
                  )
        
       
fig = tools.make_subplots(rows=1, cols=2,subplot_titles=('Subscribed/Not by number of contact', 'Ratio of y(True)/y(Total) by contacts'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3, 1, 2)

fig['layout'].update(height=600, width=800, title='Subscribed and Not Subscribed Counts and Ratio ')
iplot(fig, filename='subplot')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



Dos gráficos acima podemos ver que o número de pessoas contactadas cai quase exponencialmente com o número de ligações. Também é interessante notar que a proporção de pessoas que assinam o plano é mais elevado com apenas uma ligação, tendo uma taxa de sucesso de **14.6%**, e que depois de 4 ligações essa taxa cai para menos de **9%**. 

**Assumindo-se que o universo de pessoas que podem receber ligações é muito grande e em escala muito superior a de ligações realizadas nessa campanha (já que é uma campanha no Brasil), a taxa média e máxima de ligações indicadas é 1, pois é o número de ligações com maior taxa de sucesso e que vai permitir os atendentes atingirem o maior número de inscrições/assinaturas para o investimento proposto.**

Por exemplo, caso um grupo de funcionários consiga fazer 100.000 ligações em um dia, se eles ligarem apenas 1x para todas as pessoas, eles teriam algo em torno de *14500* sucessos da campanha no dia. Se eles ligassem no máximo 2x, eles algo perto de *12000* sucessos da campanha no dia. Quanto maior o número de ligações, pior a taxa, como podemos ver no gráfico de proporção de inscrições por contatos realizados.

Caso exista um número limitado de pessoas disponíveis para contato, um número máximo de até três contatos pode ser considerado pois em até três contatos a taxa de sucesso é de **11.9%**, não muito inferior ao melhor caso que é em uma ligação. 


<br>

### 3.4 O resultado da campanha anterior tem relevância na campanha atual?

Isolei nesse ponto o resultado da campanha anterior e o d atual:

In [19]:
poutcome_y = bank[['poutcome','y']].copy()

In [20]:
y_no_previous = poutcome_y[poutcome_y.y=='no'].poutcome
y_yes_previous = poutcome_y[poutcome_y.y=='yes'].poutcome

Gerarei dois gráficos de setores de inscritos/náo inscritos pelo resultado da última campanha:

In [21]:
trace1 = go.Pie(
    labels=y_no_previous.cat.categories.tolist(),
    values=y_no_previous.value_counts(),
    domain={"x": [0, .48]},
#    text=["lala"],
#    textinfo="text"
    name='y=0'
    
)

trace2 = go.Pie(
    labels=y_no_previous.cat.categories.tolist(),
    values=y_yes_previous.value_counts(),
    domain={"x": [.52, 1]},
    name='y=1'
)

data = [trace1,trace2]
#layout = go.Layout(
#    barmode='stack'
#)
layout = go.Layout(
    annotations=[
        dict(
            x=0.08,
            y=1.05,
            showarrow=False,
            text='Not Subscribed X Previous Campaign Result',
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.92,
            y=1.05,
            showarrow=False,
            text='Subscribed X Previous Campaign Result',
            xref='paper',
            yref='paper'
        )

    ],
    autosize=True,
    margin=dict(
        b=100
    ),
    title='Subscribed/Not Subscribed by Previous Campaign Result.',
    height=600,
    width=950
)


fig = go.Figure(data=data,layout=layout)
iplot(fig, filename='stacked-bar')

Como pode-se ver, o gráfico da esquerda representa o gráfico de não inscritos na campanha atual pelo resultado da campanha anterior. Podemos ver que desses apenas **3.84%** se inscreveram em campanhas anteriores. Já no gráfico da direita vemos que dos inscritos, **11.7%** deles se inscreveram também em campanhas anteriores, 3x mais proporcionalmente do que dos não inscritos. 

Apesar da maioria dos inscritos na campanha atual não ter se inscrito na campanha anterior, a proporção do total de inscritos em campanhas anteriores que também se inscreveu na campanha atual é pouco mais de 3x maior que a proporção de inscritos em campanhas anteriores que não se inscreveram nessa campanha. Logo, existe sim relação entre a campanha anterior e a atual.

<br>

### 3.5 Qual o fator determinante para que o banco exija um seguro de crédito?

Para exigir um fator de crédito, deve-se olhar a variável 'default'. Em finanças, 'default' é um incumprimento das condições ou obrigações legais de um empréstimo.

Para ver qual fator é determinante, usaremos o coeficiente de correlação de rank de Spearman, por ser adequado para uso em variaveis categoricas X categoricas e contínuas. 

In [22]:
for col in bank.columns:
    print(col,bank['default'].corr(bank[col],method= 'spearman'))


/home/alexandre/anaconda3/envs/dl-1.7/lib/python3.5/site-packages/scipy/stats/stats.py:245: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.



age -0.0146809906168
job -0.00525851036302
marital -0.00562699052198
education -0.0118859669748
default 1.0
balance -0.167738798338
housing -0.00602521835625
loan 0.0772342410792
contact 0.0136265867104
day 0.0119069233456
month 0.00605396082525
duration -0.00710002946216
campaign 0.0142648902159
pdays -0.0380528860956
previous -0.039278770509
poutcome 0.0386877580913
y -0.0224189658908


<br>

Podemos ver então que a variável com maior correlação com 'default' excluindo ela mesma é o balanço (*balance*), que é o saldo da conta da pessoa contactada. Essa relação me parece ser uma correlação causal, visto que um saldo negativo parece explicar de certa forma o motivo do não pagamento de uma obrigação de empréstimo.

A seguir, irei criar um subset com 'default' e 'balance' para gerar um boxplot do balanço para o caso de ter dividas em aberto ou não.

<br>

In [23]:
n_table = bank[['default','balance']].copy()

In [24]:
trace0=go.Box(y=n_table[n_table.default=="yes"].balance
             ,    name = "Balance for Default = True",
    jitter = 0.3,
    boxmean=True,
    boxpoints = False,
    marker = dict(
        color = 'rgb(7,40,89)'),
    line = dict(
        color = 'rgb(7,40,89)'),
    yaxis = "y"
)
trace1=go.Box(y=n_table[n_table.default=="no"].balance,             
           name = "Balance for Default = False",
           jitter = 0.3,
           boxmean=True,
           boxpoints = False,
           marker = dict(
           color = 'rgb(107,174,214)'),
           line = dict(
           color = 'rgb(107,174,214)'),
           yaxis = "y"
)

layout = go.Layout(
    width = 800,
    height = 500,
    
    yaxis = dict(
      anchor = "y",
      range=[-1000,4000]
    )
)
    
data = [trace0,trace1]
fig = go.Figure(data=data, layout=layout)

iplot(fig,filename='box-plot')

Desses BoxPlots podemos ver que a média e mediana do balanço de quem está com dívidas de empréstimos é negativo (-137 e -7 respectivamente), enquanto para pessoas sem dívidas, a média e mediana são ambas positivas (1390 e 468 respectivamente). Com esses dados e o dos primeiro e terceiro quartis, podemos afirmar então que o balanço deve ser um fator determinante para que o banco exija um seguro de crédito ou não.

<br>

### 3.6 Quais são as características mais proeminentes de um cliente que possua empréstimo imobiliário?

Para essa análise, usaremos um subset de variáveis que exclui todas as variáveis ligadas diretamente a campanha atual, como tipo de contato, mês, dia, duração e resultado da campanha. Fiz isso pois o empréstimo imobiliário existe desde antes da campanha atual e as variáveis selecionadas tentam explicar características do cliente com relação ao empréstimo imobiliário. 

In [25]:
housing_set = bank[['age', 'job', 'marital', 'education', 'default', 'balance', 'housing','loan']].copy()

Usaremos novamente o coeficiente de correlação de rank de Spearman pelos motivos já citados acima

In [26]:
for col in housing_set.columns:
    print(col,housing_set['housing'].corr(housing_set[col],method= 'spearman'))

/home/alexandre/anaconda3/envs/dl-1.7/lib/python3.5/site-packages/scipy/stats/stats.py:245: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.



age -0.15434020548
job -0.131364144171
marital -0.0170466399999
education -0.0943944346027
default -0.00602521835625
balance -0.0682919560213
housing 1.0
loan 0.0413228660413


Dessa análise, pode-se ver que as variáveis com maior correlação de clientes com empréstimo imobiliário são a idade e profissão. Outras variáveis serão printadas individualmente abaixo para se comparar com quem não tem empréstimo imobiliario:

In [27]:
print(housing_set[housing_set['housing']=='yes'].education.value_counts())
print(housing_set[housing_set['housing']=='yes'].default.value_counts())
print(housing_set[housing_set['housing']=='yes'].marital.value_counts())
print(housing_set[housing_set['housing']=='yes'].loan.value_counts())

secondary    14038
tertiary      6378
primary       3894
unknown        820
Name: education, dtype: int64
no     24695
yes      435
Name: default, dtype: int64
married     15321
single       6902
divorced     2907
Name: marital, dtype: int64
no     20763
yes     4367
Name: loan, dtype: int64


In [28]:
print(housing_set[housing_set['housing']=='no'].education.value_counts())
print(housing_set[housing_set['housing']=='no'].default.value_counts())
print(housing_set[housing_set['housing']=='no'].marital.value_counts())
print(housing_set[housing_set['housing']=='no'].loan.value_counts())

secondary    9164
tertiary     6923
primary      2957
unknown      1037
Name: education, dtype: int64
no     19701
yes      380
Name: default, dtype: int64
married     11893
single       5888
divorced     2300
Name: marital, dtype: int64
no     17204
yes     2877
Name: loan, dtype: int64


Podemos ver que o perfil de quem tem empréstimo imobiliário é de educação secundária, sem descumprimento de empréstimos, casado e sem empréstimo financeiro. É possível ver também que quem não tem empréstimo imobiliário também possui tais características, apesar de com proporções um pouco diferenciadas nas variáveis de educação e matrimonial.

Analisarei individualmente as características de variáveis numéricas a seguir



**Profissão:**


In [29]:
job_dummy = pd.get_dummies(housing_set[["job", "housing"]], columns=["housing"])
job_d_agg = job_dummy.groupby('job').sum()

In [30]:
data = [go.Bar(x=job_d_agg.index.get_level_values(0),
            y=job_d_agg.housing_no,name='No Housing Loan'),
       go.Bar(x=job_d_agg.index.get_level_values(0),
            y=job_d_agg.housing_yes,name='Has Housing Loan'),
       ]
layout = go.Layout(
    title='People with Housing Loan by Jobs',
)
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='agg_f-plot')

Do gráfico acima podemos ver que trabalhadores de colarinho azul (*blue-collar*) são os que possuem mais empréstimo imobiliário em números absolutos e proporcionalmente, logo, pode-se dizer que essa é uma característica proeminente de clientes que possuem empréstimos imobiliários.



**Idade:**

In [31]:
trace0=go.Box(y=housing_set[housing_set['housing']=='no'].age
             ,    name = "No Housing Loan"
)
trace1=go.Box(y=housing_set[housing_set['housing']=='yes'].age,             
           name = "Has Housing Loan"
)
layout = go.Layout(
    title='People with Housing Loan by Age',
)
    
data = [trace0,trace1]
fig = go.Figure(data=data, layout=layout)

iplot(fig,filename='box-plot-housing')

Desse gráfico também é possível perceber que apesar da primeiro quartil quase iguais, na mediana a diferença aumenta e no terceiro quartil se distancia ainda mais. Logo, apesar da proporção até 32~33 anos ser similar nos dois casos, a população de quem tem empréstimo imobiliário é em termos gerais mais jovem do que a que não tem empréstimo imobiliário, o que faz sentido pois depois de certa idade, espera-se que a pessoa já tenha quitado seu empréstimo imobiliário.

**Balanço:**

In [32]:
trace0=go.Box(y=housing_set[housing_set['housing']=='no'].balance
             ,    name = "No Housing Loan",
           boxpoints = False

)
trace1=go.Box(y=housing_set[housing_set['housing']=='yes'].balance,             
           name = "Has Housing Loan",
           boxpoints = False

)
layout = go.Layout(
    title='People with Housing Loan by Balance',
    width = 800,
    height = 500,
    
    yaxis = dict(
      anchor = "y",
      range=[-1000,4000]
    )
)
    
data = [trace0,trace1]
fig = go.Figure(data=data, layout=layout)

iplot(fig,filename='box-plot-housing-2')

O balanço teve variação com mediana 23% maior em quem não tem que empréstimo imobiliário do que quem tem empréstimo imobiliário. Logo, em termos gerais, também pode-se afirmar que quem não tem empréstimo imobiliário tem uma conta com mais saldo do que quem paga empréstimo imobiliário.  